In [2]:
import os
import pickle
import torch
import torch.nn as nn
import torch.utils.data
import torchvision.datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime
from tqdm import tqdm
from copy import deepcopy
from cifar10_utils import get_cifar10, get_dataloader

from models_v2 import (BasicCNN, AutoencoderCNN, ECACNN, AutoencoderECACNN,
                       ECASpatialCNN, DeeperCNN, LinearAutoencoderECACNN, CBAMCNN)

In [3]:
# Seed for reproduceability
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False
np.random.seed(42)

# Setup device-agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
cifar10 = get_cifar10()
cifar10_loader = get_dataloader(cifar10, 128)

100%|██████████| 170498071/170498071 [00:12<00:00, 13133635.04it/s]


Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


In [5]:
def evaluate_model(model, data_loader):
    """
    Performs the evaluation of the MLP model on a given dataset.

    Args:
      model: An instance of 'MLP', the model to evaluate.
      data_loader: The data loader of the dataset to evaluate.
    Returns:
        accuracy
    """
    accuracies_per_batch, losses_per_batch = [], []
    loss_module = nn.CrossEntropyLoss()
    # Get accuracy for epoch
    for batch in data_loader:

        # Get validation images and labels
        X = batch[0].to(device)
        y = batch[1].to(device)

        # Get predictions on validation set
        model.eval()
        with torch.no_grad():
            pred_logits = model.forward(X)
            pred_classes = torch.argmax(torch.softmax(pred_logits, dim=1), axis=1)

        # Calculate accuracy := # of correct preds / total # of preds
        current_accuracy = torch.sum(pred_classes == y) / pred_classes.shape[0]
        accuracies_per_batch.append(current_accuracy.item())
        current_loss = loss_module(pred_logits, y).item()
        losses_per_batch.append(current_loss)

    accuracy = np.average(accuracies_per_batch)
    loss = np.average(losses_per_batch)

    return accuracy, loss

In [6]:
def train(model, epochs=10, lr=0.1, momentum=0, verbose=True):

    logging_dict = {'loss': {'train': [], 'validation': []},
                    'accuracy': {'train': [], 'validation': []},
                    'lr': [],
                    'batches_per_epoch': [],
                    'momentum': momentum}

    for epoch in tqdm(range(epochs)):

        batches_per_epoch = 0

        model.train()

        # Loss module and optimizer
        loss_module = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1)


        for batch in cifar10_loader['train']:

            batches_per_epoch += 1

            # Get training images and labels
            X_train = batch[0].to(device)
            y_train = batch[1].to(device)

            # Forward pass
            train_pred_logits = model.forward(X_train)

            # Calculate loss
            loss = loss_module(train_pred_logits, y_train)
            logging_dict['loss']['train'].append(loss.item())

            # Calculate accuracy
            train_pred_class = torch.argmax(torch.softmax(train_pred_logits, dim=1), axis=1)
            train_accuracy = torch.sum(train_pred_class == y_train) / train_pred_class.shape[0]
            logging_dict['accuracy']['train'].append(train_accuracy.item())

            # Zero gradients
            optimizer.zero_grad()

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer.step()


        # Log num of batches for this epoch
        logging_dict['batches_per_epoch'].append(batches_per_epoch)

        # Log current LR
        logging_dict['lr'].append(optimizer.param_groups[0]['lr'])


        # Get metrics on validation set
        validation_accuracy, validation_loss = evaluate_model(model, cifar10_loader['validation'])

        # Update LR
        scheduler.step(validation_loss)

        # Determine if best model
        if len(logging_dict['accuracy']['validation']) == 1 or \
            all([validation_accuracy > acc for acc in logging_dict['accuracy']['validation']]):
            best_model = deepcopy(model)

        logging_dict['accuracy']['validation'].append(validation_accuracy.item())
        logging_dict['loss']['validation'].append(validation_loss.item())

        if verbose:
            print(f'\n{epoch = }, '
                  f'training accuracy: {train_accuracy.item():.3f}, '
                  f'training loss: {loss.item():.3f}',
                  f'validation accuracy: {validation_accuracy.item():.3f}, '
                  f'validation loss: {validation_loss.item():.3f}',
                 )

    # Get metrics on test set
    test_accuracy, test_loss = evaluate_model(best_model, cifar10_loader['test'])
    if verbose:
        print(f'test accuracy: {test_accuracy}, test loss: {test_loss}')

    return best_model, test_accuracy, test_loss, logging_dict

In [ ]:
num_runs = 5

results = {'cnn': [],
            'eca': [],
            'cbam': [],
            'autoencoder': [],
            'autoencoder_eca': [],
            'linear_autoencoder_eca': [],
            'eca_spatial': [],
            'deeper_cnn': [],
}

for run in range(num_runs):
    cnn_model = BasicCNN().to(device)
    autoencoder_model = AutoencoderCNN().to(device)
    eca_model = ECACNN().to(device)
    autoencoder_eca_model = AutoencoderECACNN().to(device)
    eca_spatial_model = ECASpatialCNN().to(device)
    deeper_cnn_model = DeeperCNN().to(device)
    linear_autoencoder_model = LinearAutoencoderECACNN().to(device)
    cbam_model = CBAMCNN().to(device)


    models = {
        'cnn': cnn_model,
        'eca': eca_model,
        'cbam': cbam_model,
        'autoencoder': autoencoder_model,
        'autoencoder_eca': autoencoder_eca_model,
        'linear_autoencoder_eca': linear_autoencoder_model,
        'eca_spatial': eca_spatial_model,
        'deeper_cnn': deeper_cnn_model,
    }

    # Kaiming initialization
    def init_weights(m):
        if hasattr(m, 'weight') and m.weight is not None and len(m.weight.shape) > 1:
            nn.init.kaiming_normal_(m.weight)
        if hasattr(m, 'bias') and m.bias is not None:
            nn.init.zeros_(m.bias)
    for model in models.values():
        model.apply(init_weights)


    print(f'run {run + 1} / {num_runs}')
    for name, model in models.items():
        print(f'training {name}')
        best_model, test_accuracy, test_loss, logging_dict = train(model, verbose=False, epochs=10)
        results[name].append(
            {name: {'best_model': best_model.state_dict(),
            'test_accuracy': test_accuracy,
            'test_loss': test_loss,
            'logging_dict': logging_dict}})
        print('done training.')